In [ ]:
import os
from getpass import getpass
from google import genai

# pass your API key here
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY") or getpass(
    "Enter Google API Key: "
)
# initialize our client
client = genai.Client()

In [2]:
from IPython.display import Markdown

model_id = "gemini-2.0-flash-exp"

response = client.models.generate_content(
    model=model_id, contents='What is Gemini?'
)
Markdown(response.text)

Gemini refers to a few different things, but most commonly it refers to **Google's large language model.** Here's a breakdown:

**1. Gemini (The Google AI Model):**

*   **What it is:** Gemini is a multimodal AI model developed by Google AI. It's designed to be Google's most capable and general AI model, surpassing previous models like LaMDA and PaLM.
*   **Capabilities:** It can understand and generate text, images, audio, video, and code. This "multimodal" nature means it can process and understand information from various sources simultaneously. It's designed to be highly adaptable and perform well on a wide range of tasks.
*   **Versions:** There are different versions of Gemini optimized for various use cases:
    *   **Gemini Ultra:** The most powerful version, designed for highly complex tasks.
    *   **Gemini Pro:** A more balanced version, suitable for a wide range of applications, including powering Google's Bard (now Gemini) chatbot.
    *   **Gemini Nano:** A lightweight version designed to run on devices like smartphones and other edge devices.
*   **Use Cases:** Gemini is being integrated into various Google products and services, including:
    *   **Search:** Improving search results and providing more comprehensive answers.
    *   **Gmail & Workspace:** Helping with email composition, summarizing documents, and generating ideas.
    *   **Bard/Gemini Chatbot:** Providing more natural and helpful conversational AI.
    *   **Coding:** Assisting developers with code generation, debugging, and understanding code.
    *   **Creative Content:** Helping with writing, image generation, and other creative tasks.

**2. Gemini (The Zodiac Sign):**

*   In astrology, Gemini is the third sign of the zodiac, associated with people born between approximately May 21 and June 20. It's an air sign often associated with communication, curiosity, and adaptability.

**3. Project Gemini (NASA):**

*   In the 1960s, Project Gemini was a NASA human spaceflight program that served as a bridge between Project Mercury (the first US human spaceflight program) and Project Apollo (the program that landed humans on the Moon). Gemini developed techniques for rendezvous, docking, and spacewalking that were crucial for the success of Apollo.

**In summary, when someone mentions "Gemini" today, they are most likely referring to the Google AI model.**  To avoid confusion, it's helpful to consider the context in which the word is used.


In [4]:
from google.genai.types import Tool, GoogleSearch

search_tool = Tool(google_search=GoogleSearch())

In [ ]:
from google.genai.types import GenerateContentConfig
model_id = "gemini-2.0-flash-exp"

config = GenerateContentConfig(
    system_instruction=(
        "You are a helpful assistant that provides up to date information "
        "to help the user in their research."
    ),
    tools=[search_tool],
    response_modalities=["TEXT"],
    temperature=0.0,  # likely default
    candidate_count=1,  # likely default

)
config
response = client.models.generate_content(
    model=model_id,
    contents="Tell me the latest news in AI",
    config=config
)
Markdown(response.text)

GenerateContentConfig(http_options=None, system_instruction='You are a helpful assistant that provides up to date information to help the user in their research.', temperature=0.0, top_p=None, top_k=None, candidate_count=1, max_output_tokens=None, stop_sequences=None, response_logprobs=None, logprobs=None, presence_penalty=None, frequency_penalty=None, seed=None, response_mime_type=None, response_schema=None, routing_config=None, model_selection_config=None, safety_settings=None, tools=[Tool(retrieval=None, google_search=GoogleSearch(), google_search_retrieval=None, enterprise_web_search=None, google_maps=None, code_execution=None, function_declarations=None)], tool_config=None, labels=None, cached_content=None, response_modalities=['TEXT'], media_resolution=None, speech_config=None, audio_timestamp=None, automatic_function_calling=None, thinking_config=None)

In [ ]:
response = client.models.generate_content(
    model=model_id,
    contents="Tell me the latest news in AI",
    config=config
)
Markdown(response.text)

In [9]:
response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents="Tell me the latest news in AI",
    config={
        "tools": [{"google_search": {}}],
        "temperature": 0,
        },
    )
Markdown(response.text)

KeyboardInterrupt: 

In [140]:
from dataclasses import Field
from langchain_tavily import TavilySearch
from langgraph.prebuilt import create_react_agent
from langchain_deepseek import ChatDeepSeek
from dotenv import load_dotenv
from typing import TypedDict, List, Annotated, Literal, Dict, Union, Optional 
from pydantic import BaseModel, Field

class Citation(BaseModel):
    label: str = Field(
        ...,
        description="The name of the website.",
    )
    source_id: str = Field(
        ...,
        description="The url of a SPECIFIC source which justifies the answer.",
    )
    quote: str = Field(
        ...,
        description="The VERBATIM quote from the specified source that justifies the answer.",
    )


class QuotedAnswer(BaseModel):
    """Answer the user question based only on the given sources, and cite the sources used."""
    answer: str = Field(
        ...,
        description="The answer to the user question, which is based only on the given sources. Include any relevant sources in the answer as markdown hyperlinks. For example: 'This is a sample text ([url website](url))'"
    )
    citations: List[Citation] = Field(
        ..., description="Citations from the given sources that justify the answer."
    )

load_dotenv()
llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=1,
    timeout=None,
    max_retries=2,
    )


# Initialize Tavily Search Tool
tavily_search_tool = TavilySearch(
    max_results=5,
    topic="general",
)
structured_llm = llm.bind_tools([tavily_search_tool]).with_structured_output(QuotedAnswer)
# agent = create_react_agent(llm, [tavily_search_tool])

user_input = "What nation hosted the Euro 2024?"
# formatted_prompt = web_searcher_instructions.format(
#     current_date=get_current_date(),
#     research_topic='price of 5090',
# )
formatted_prompt = web_searcher_instructions.format(
    current_date=get_current_date(),
    research_topic=user_input,
) + "\n\nIMPORTANT: Use the search tool provided to find the most up-to-date information."


# for step in agent.stream(
#     {"messages": user_input},
#     stream_mode="values",
# ):
#     step["messages"][-1].pretty_print()
result = structured_llm.invoke(formatted_prompt)
print(result)
# https://github.com/tavily-ai/use-cases/blob/main/company-research/company_research.ipynb

answer='The UEFA Euro 2024 was hosted by Germany. The tournament took place from June 14 to July 14, 2024, across various cities in Germany.' citations=[Citation(label='UEFA Official Website', source_id='https://www.uefa.com/uefaeuro-2024/', quote='Germany will host the UEFA Euro 2024, with matches scheduled from June 14 to July 14, 2024.'), Citation(label='BBC Sport', source_id='https://www.bbc.com/sport/football', quote='Germany was selected as the host nation for Euro 2024, marking the first time the country has hosted the tournament since 1988.')]


In [136]:
from langchain_tavily import TavilySearch
from langgraph.prebuilt import create_react_agent
from langchain_deepseek import ChatDeepSeek
from dotenv import load_dotenv
from backend.src.agent.prompts import (
    get_current_date,
    query_writer_instructions,
    web_searcher_instructions,
    reflection_instructions,
    answer_instructions,
)

load_dotenv()
llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=1,
    timeout=None,
    max_retries=2,
    )
# Create a ReAct agent with the web searcher system prompt and Tavily search tool
formatted_prompt = web_searcher_instructions.format(
    current_date=get_current_date(),
    research_topic='price of 5090',
)
tavily_search_tool = TavilySearch(
    max_results=5,
    topic="general",
)

# Create the agent with the formatted prompt as system message
agent = create_react_agent(
    llm,
    [tavily_search_tool],
    
)
prompt=formatted_prompt
# Create a runnable chain from the agent
# search_chain = agent.get_chain()

# agent.invoke({"messages": [HumanMessage(content="What is the price of 5090?")]}).pretty_print()

# user_input = "What is the price of 5090?"
# response = agent.invoke({"messages": prompt})
for step in agent.stream(
    {"messages": prompt},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()


================================ Human Message =================================

Conduct targeted Google Searches to gather the most recent, credible information on "price of 5090" and synthesize it into a verifiable text artifact.

Instructions:
- Query should ensure that the most current information is gathered. The current date is June 13, 2025.
- Conduct multiple, diverse searches to gather comprehensive information.
- Consolidate key findings while meticulously tracking the source(s) for each specific piece of information.
- The output should be a well-written summary or report based on your search findings. 
- Only include the information found in the search results, don't make up any information.

Research Topic:
price of 5090

================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_0_6abbc5f6-fc30-455c-b0fc-453678cfd960)
 Call ID: call_0_6abbc5f6-fc30-455c-b0fc-453678cfd960
  Args:
    query: price of 5090 June 2025
  

In [139]:
step

{'messages': [HumanMessage(content='Conduct targeted Google Searches to gather the most recent, credible information on "price of 5090" and synthesize it into a verifiable text artifact.\n\nInstructions:\n- Query should ensure that the most current information is gathered. The current date is June 13, 2025.\n- Conduct multiple, diverse searches to gather comprehensive information.\n- Consolidate key findings while meticulously tracking the source(s) for each specific piece of information.\n- The output should be a well-written summary or report based on your search findings. \n- Only include the information found in the search results, don\'t make up any information.\n\nResearch Topic:\nprice of 5090\n', additional_kwargs={}, response_metadata={}, id='25a1015f-d63a-4ae0-a427-adeef3aa5299'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0_6abbc5f6-fc30-455c-b0fc-453678cfd960', 'function': {'arguments': '{"query": "price of 5090 June 2025", "search_depth": "advan

In [90]:
from IPython.display import Markdown

Markdown(response['messages'][4].content)

{"query": "latest updates on RTX 5090 pricing June 2025", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://x.com/USRestocks/status/1932361271782863013", "title": "US Restocks RTX5090/60/60Ti § Switch2", "content": "RTX 5090 in Stock! | Cart Link: https://amzn.to/4bPc14X June 10, 2025 at 01:56AM PT #ad MSI Gaming RTX 5090 32G Gaming Trio OC Graphics Card (32GB GDDR7,", "score": 0.79254496, "raw_content": null}, {"url": "https://www.youtube.com/watch?v=l641VkblkfY", "title": "Best GPUs to Buy for Gaming RIGHT NOW! [June/July 2025]", "content": "Newegg (US) [PAID LINKS]:\nB570: https://newegg.io/nc0f72caf3\nB580: https://newegg.io/ncb8aa2818\nRTX 5060: https://newegg.io/nc2d2d6cb4\nRTX 5060 Ti 16GB: https://newegg.io/nce151063f\nRX 9070: https://newegg.io/nc6c904af2\nRTX 5070 Ti: https://newegg.io/ncc8d843aa\nRX 9070 XT: https://newegg.io/ncd7924039\nRTX 5080: https://newegg.io/nc45487d71\nRTX 5090: https://newegg.io/nc60c8042e [...] Amazon (UK) [PAID LINKS]:\nB570: https://amzlink.to/az0zJQpeqq2Js\nB580: https://amzlink.to/az0LAlIxFBfKo\nRTX 5060: https://amzlink.to/az0hIVbEZrI6B\nRTX 5060 Ti 16GB: https://amzlink.to/az0HkaR5MIpvR\nRX 9070: https://amzlink.to/az0syS72HTuPw\nRTX 5070 Ti: https://amzlink.to/az0pZ6PXayEAU\nRX 9070 XT: https://amzlink.to/az0LwqrQy9bK1\nRTX 5080: https://amzlink.to/az0n4hQDRq9rx\nRTX 5090: https://amzlink.to/az0HXuNlzKZWX [...] Amazon (US) [PAID LINKS]:\nB570: https://amzlink.to/az0HOxxNV4OF8\nB580: https://amzlink.to/az0u4cH6r0fB2\nRTX 5060: https://amzlink.to/az0zbM77NOV2Z\nRTX 5060 Ti 16GB: https://amzlink.to/az0YZkOnMFXY3\nRX 9070: https://amzlink.to/az03UwkWLMGmH\nRTX 5070 Ti: https://amzlink.to/az0AItZVPbPNO\nRX 9070 XT: https://amzlink.to/az0oOI9UxwEay\nRTX 5080: https://amzlink.to/az0SSNK8AQuzo\nRTX 5090: https://amzlink.to/az0aj8ecrctEC", "score": 0.73023266, "raw_content": null}, {"url": "https://en.gamegpu.com/iron/msi-and-gigabyte-are-delaying-rtx-5090-deliveries-to-susa-until-power-rates-are-up-by-july-9", "title": "MSI and Gigabyte RTX 5090 Shipments to US Ahead of Tariffs ...", "content": "# MSI and Gigabyte RTX 5090 Shipments to US Ahead of Tariffs Coming into Effect on July 9\n\nAuthor [Maximum Games](https://en.gamegpu.com/view-user-profile?user=41287), 10 2025 June. Published in [Hardware](/backgammon/news/iron/) [...] ![rtx 5090 gigabyte](https://gamegpu.com/images/1_2025/NEWS/q2/f123/9600/rtx_5090_gigabyte.jpg)\n\n![rtx 5090 gigabyte](https://gamegpu.com/images/1_2025/NEWS/q2/f123/9600/rtx_5090_gigabyte.jpg) [...] Experts suggest that **NVIDIA's AIB Partners Will Hold Back RTX 5090 Stock**, waiting for the official return of tariffs. This will give the opportunity to raise prices after the deadline if agreements between the countries are not reached. A similar scheme was used in 2016-2017, when companies massively imported video cards before the start of trade restrictions. **RTX 5090 Availability May Worsen in July**, especially if tariffs are reintroduced and companies decide to pass the costs on to", "score": 0.65173227, "raw_content": null}, {"url": "https://www.notebookcheck.net/2025-Lenovo-Legion-Pro-7i-with-RTX-5090-and-Core-Ultra-9-275HX-gets-another-price-cut.1033540.0.html", "title": "2025 Lenovo Legion Pro 7i with RTX 5090 and Core Ultra 9 275HX ...", "content": "Deal | 2025 Lenovo Legion Pro 7i with RTX 5090 and Core Ultra 9 275HX gets another price cut ... It goes without saying that the top-end RTX 5090 configuration of", "score": 0.64719695, "raw_content": null}, {"url": "https://www.tomshardware.com/pc-components/gpus/cheapest-5090-ever-lucky-pc-builder-finds-rtx-5090-for-just-usd1-679-at-walmart-saving-usd1-520-card-has-worked-flawlessly-ever-since", "title": "Cheapest 5090 ever? Lucky PC builder finds RTX 5090 for 'just ...", "content": "Redditor Sufficient\\_Crazy7758 took to the platform to hail their purchase of the \"Cheapest 5090 ever?\", after scooping a PNY GeForce RTX 5090 from Walmart for the relatively low price of $1,679.99. The attached images reveal that the card was discounted by over $1,500 on May 25, having previously been discounted on May 13.\n\n\"Just so all of you know, I bought this on that date and it has worked flawlessly ever since,\" they said triumphantly. [...] In the ever-evolving world of insane GPU prices, tariffs, and frightful stock shortages, one intrepid PC builder has managed to swipe an Nvidia RTX 5090 from Walmart for \"just\" $1,679, more than $200 below MSRP. [...] even the [Founders Edition still retails for the same price](https://goto.walmart.com/c/1943169/565706/9383?subId1=tomshardware-us-9572975325724004555&sharedId=tomshardware-us&u=https%3A%2F%2Fwww.walmart.com%2Fip%2FNVIDIA-GeForce-RTX-5090-Graphic-Card-32GB-GDDR7%2F15176552726%3FclassType%3DREGULAR%26from%3D%2Fsearch), way over the $1,999 MSRP.", "score": 0.6016175, "raw_content": null}], "response_time": 2.98}

In [47]:
user_input = "What is the price of 5090?"

for step in agent.stream(
    {"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the price of 5090?
================================== Ai Message ==================================


In [110]:
import os

from backend.src.agent.tools_and_schemas import SearchQueryList, Reflection
from dotenv import load_dotenv
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.types import Send
from langgraph.graph import StateGraph
from langgraph.graph import START, END
from langchain_core.runnables import RunnableConfig
from google.genai import Client

from backend.src.agent.state import (
    OverallState,
    QueryGenerationState,
    ReflectionState,
    WebSearchState,
)
from backend.src.agent.configuration import Configuration
from backend.src.agent.prompts import (
    get_current_date,
    query_writer_instructions,
    web_searcher_instructions,
    reflection_instructions,
    answer_instructions,
)
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_deepseek import ChatDeepSeek

from backend.src.agent.utils import (
    get_citations,
    get_research_topic,
    insert_citation_markers,
    resolve_urls,
)


In [111]:

def generate_query(state: OverallState, config: RunnableConfig) -> QueryGenerationState:
    """LangGraph node that generates a search queries based on the User's question.

    Uses Gemini 2.0 Flash to create an optimized search query for web research based on
    the User's question.

    Args:
        state: Current graph state containing the User's question
        config: Configuration for the runnable, including LLM provider settings

    Returns:
        Dictionary with state update, including search_query key containing the generated query
    """
    configurable = Configuration.from_runnable_config(config)

    # check for custom initial search query count
    if state.get("initial_search_query_count") is None:
        state["initial_search_query_count"] = configurable.number_of_initial_queries

    # init Gemini 2.0 Flash
    # llm = ChatGoogleGenerativeAI(
    #     model=configurable.query_generator_model,
    #     temperature=1.0,
    #     max_retries=2,
    #     api_key=os.getenv("GEMINI_API_KEY"),
    # )
    llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=1,
    timeout=None,
    max_retries=2,
    )
    structured_llm = llm.with_structured_output(SearchQueryList)
    print(get_research_topic(state["messages"]),)
    # Format the prompt
    current_date = get_current_date()
    formatted_prompt = query_writer_instructions.format(
        current_date=current_date,
        research_topic=get_research_topic(state["messages"]),
        number_queries=state["initial_search_query_count"],
    )
    # Generate the search queries
    result = structured_llm.invoke(formatted_prompt)
    print(result)
    return {"query_list": result.query}

In [112]:
def continue_to_web_research(state: QueryGenerationState):
    """LangGraph node that sends the search queries to the web research node.

    This is used to spawn n number of web research nodes, one for each search query.
    """
    return [
        Send("web_research", {"search_query": search_query, "id": int(idx)})
        for idx, search_query in enumerate(state["query_list"])
    ]


def web_research(state: WebSearchState, config: RunnableConfig) -> OverallState:
    """LangGraph node that performs web research using the native Google Search API tool.

    Executes a web search using the native Google Search API tool in combination with Gemini 2.0 Flash.

    Args:
        state: Current graph state containing the search query and research loop count
        config: Configura"tion for the runnable, including search API settings

    Returns:
        Dictionary with state update, including sources_gathered, research_loop_count, and web_research_results
    """
    # Configure
    configurable = Configuration.from_runnable_config(config)
    formatted_prompt = web_searcher_instructions.format(
        current_date=get_current_date(),
        research_topic=state["search_query"],
    )
    llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=1,
    timeout=None,
    max_retries=2,
    )

    # Create the agent with the formatted prompt as system message
    # agent = create_react_agent(
    #     llm,
    #     [tavily_search_tool],
    #     prompt=formatted_prompt
    # )
    # Uses the google genai client as the langchain client doesn't return grounding metadata
    genai_client = Client(api_key=os.getenv("GEMINI_API_KEY"))

    response = genai_client.models.generate_content(
        model=configurable.query_generator_model,
        contents=formatted_prompt,
        config={
            "tools": [{"google_search": {}}],
            "temperature": 0,
        },
    )
    print('search results:')

    print(response)
    
    # resolve the urls to short urls for saving tokens and time
    resolved_urls = resolve_urls(
        response.candidates[0].grounding_metadata.grounding_chunks, state["id"]
    )
    # Gets the citations and adds them to the generated text
    citations = get_citations(response, resolved_urls)
    modified_text = insert_citation_markers(response.text, citations)
    sources_gathered = [item for citation in citations for item in citation["segments"]]

    return {
        "sources_gathered": sources_gathered,
        "search_query": [state["search_query"]],
        "web_research_result": [modified_text],
    }

In [113]:
builder = StateGraph(OverallState)

# Define the nodes we will cycle between
builder.add_node("generate_query", generate_query)
builder.add_node("web_research", web_research)

builder.add_edge(START, "generate_query")
builder.add_conditional_edges(
    "generate_query", continue_to_web_research, ["web_research"]
)
builder.add_edge("web_research", END)

graph = builder.compile(name="pro-search-agent")

In [114]:
# graph.invoke('predict the tendency of bitcoin in the next 6 months')
state = graph.invoke({"messages": [{"role": "user", "content": "Who won the euro 2024"}], "max_research_loops": 3, "initial_search_query_count": 3})

Who won the euro 2024
query=['Euro 2024 winner'] rationale='The query is designed to retrieve the most current and accurate information about the winner of the Euro 2024 tournament, ensuring the result is specific and up-to-date as of June 2025.'
search results:
candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text="Based on the search results, here's a summary of the Euro 2024 winner:\n\n**Spain won Euro 2024**, defeating England 2-1 in the final held in Berlin on July 14, 2024. This victory marks Spain's record-breaking fourth European Championship title. Spain won all seven of their matches during the tournament. England has now lost back-to-back Euro finals. The final score was 2-1. Nico Williams and Mikel Oyarzabal scored for Spain in the final.\n")], role='model'), citation_metadata=None, finish_message=None, token_cou

In [115]:
state 

{'messages': [HumanMessage(content='Who won the euro 2024', additional_kwargs={}, response_metadata={}, id='82ff8f15-e665-478c-b075-871fa5ab10b8')],
 'search_query': ['Euro 2024 winner'],
 'web_research_result': ["Based on the search results, here's a summary of the Euro 2024 winner:\n\n**Spain won Euro 2024**, defeating England 2-1 in the final held in Berlin on July 14, 2024. [olympics](https://vertexaisearch.cloud.google.com/id/0-0) [youtube](https://vertexaisearch.cloud.google.com/id/0-1) [uefa](https://vertexaisearch.cloud.google.com/id/0-2) [uefa](https://vertexaisearch.cloud.google.com/id/0-3) This victory marks Spain's record-breaking fourth European Championship title. [uefa](https://vertexaisearch.cloud.google.com/id/0-3) [aljazeera](https://vertexaisearch.cloud.google.com/id/0-4) [youtube](https://vertexaisearch.cloud.google.com/id/0-1) [youtube](https://vertexaisearch.cloud.google.com/id/0-5) [wikipedia](https://vertexaisearch.cloud.google.com/id/0-6) [wikipedia](https://ve

In [ ]:
builder = StateGraph(OverallState, config_schema=Configuration)

# Define the nodes we will cycle between
builder.add_node("generate_query", generate_query)
builder.add_node("web_research", web_research)
builder.add_edge(START, "generate_query")
builder.add_conditional_edges(
    "generate_query", continue_to_web_research, ["web_research"]
)

builder.add_edge("finalize_answer", END)

graph = builder.compile(name="pro-search-agent")


In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import AIMessage, HumanMessage

class LLMGeneratedSearchQueries(BaseModel):
    """Schema for the LLM to generate a list of search queries."""
    search_queries: list[str] = Field(description="A list of search queries to find relevant information based on the research topic.")

llm = ChatDeepSeek(
    model="deepseek-chat",
        temperature=0.0,
        max_tokens=1024,
        timeout=None,
        max_retries=2,
    )

messages = [HumanMessage(content=formatted_prompt)]

query_generation_response = llm.invoke(
    messages,
    tools=[LLMGeneratedSearchQueries]
)

generated_queries = []
if query_generation_response.tool_calls:
    for tool_call in query_generation_response.tool_calls:
        if tool_call['name'] == LLMGeneratedSearchQueries.__name__:
            generated_queries.extend(tool_call['args'].get('search_queries', []))

TypeError: argument of type 'ModelMetaclass' is not iterable